In [2]:
#Upload your dataset File
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset (1).csv


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [3]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
#Importing Required Libraries
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from google.colab import files


In [6]:
#Read file
df = pd.read_csv('dataset.csv')
#UNIX Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv('parking_stream.csv', index=False)


In [7]:
#  Define Pathway Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int


In [8]:
#Load Streaming Data
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [9]:
# Parse & enrich
data_with_time = data.with_columns(
    timestamp_parsed = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    occ = data.Occupancy,
    cap = data.Capacity
).filter(
    (pw.this.occ.is_not_none()) &
    (pw.this.cap.is_not_none()) &
    (pw.this.timestamp_parsed.is_not_none())
)


In [10]:
#Demand function
BASE_PRICE = 10.0
ALPHA = 2.0

micro = (
    data_with_time.windowby(
        pw.this.timestamp_parsed,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30))
    )
    .reduce(
        occ_sum = pw.reducers.sum(pw.this.occ),
        occ_count = pw.reducers.count(1),
        cap_max = pw.reducers.max(pw.this.cap),
        t = pw.reducers.max(pw.this.timestamp_parsed)
    )
    .with_columns(
        price = BASE_PRICE + ALPHA * ((pw.this.occ_sum / pw.this.occ_count) )
    )
)


In [11]:
# Real-time Bokeh plot
pn.extension()  # Needed for Bokeh + Panel in Colab

def model1_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 1: Real-Time Price", x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.circle("t", "price", source=source, size=6, color="black")
    return fig

viz = micro.plot(model1_plotter, sorting_col="t")
pn.Column(viz).servable()
#pn.serve(pn.Column(viz), show=True, port=5006)

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [12]:
pw.io.csv.write(micro, "model1_output.csv")


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [13]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()